# How to update the nft metadata

You can update a nft metadata only if the policy allow you to mint or burn the token.

### 1) Create the NFT

In [95]:
%%bash
cardano-cli query utxo \
--address $(cat /users/$(whoami)/testnet/priv/wallet/Lola/lola.addr) \
--testnet-magic 1 

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
98b172afe15ce5b0efb28bd604431cdd23f8351f90271415f86e782087337872     1        9500000000 lovelace + TxOutDatumNone


In [96]:
%%bash
# Create the mint and burn keys
cardano-cli address key-gen \
--verification-key-file priv/NFT/nft.vkey \
--signing-key-file priv/NFT/nft.skey

# Create the policy script
NFT_VKEY=$(cardano-cli address key-hash --payment-verification-key-file priv/NFT/nft.vkey)
SCRIPT_PATH=policy/nft.script
# Script locked after one year
SLOTS=31536000
TIME_CONSTRAINT=$(($(cardano-cli query tip --testnet-magic 1 | jq .slot) + SLOTS))

# Generate the policy script
echo "{" >> $SCRIPT_PATH
echo "  \"type\": \"all\"," >> $SCRIPT_PATH
echo "  \"scripts\":" >> $SCRIPT_PATH
echo "  [" >> $SCRIPT_PATH
echo "    {" >> $SCRIPT_PATH
echo "      \"type\":\"before\"," >> $SCRIPT_PATH
echo "      \"slot\":$TIME_CONSTRAINT" >> $SCRIPT_PATH
echo "    }," >> $SCRIPT_PATH
echo "    {" >> $SCRIPT_PATH
echo "      \"type\": \"sig\"," >> $SCRIPT_PATH
echo "      \"keyHash\": \"$NFT_VKEY\"" >> $SCRIPT_PATH
echo "    }" >> $SCRIPT_PATH
echo "  ]" >> $SCRIPT_PATH
echo "}" >> $SCRIPT_PATH

# Create the policy id
cardano-cli transaction policyid \
--script-file policy/nft.script \
> policy/nft.policyid

# Create the metadata
NFT_ID=721
POLICY_ID=$(cat policy/nft.policyid)
ASSET_NAME="3DCN3986"
IMAGE="ipfs://QmVYfnHBhmpVKjAHRpxtwt6XThnafKW37tFtxvCbmHGjzN"
DESCRIPTION="3D render of the clay nation NFT 3986 generate with DALL-E-2 AI"
NFT_METADATA_PATH=metadata/nft.metadata

echo "{" >> $NFT_METADATA_PATH
echo "  \"$NFT_ID\": {" >> $NFT_METADATA_PATH
echo "    \"$POLICY_ID\": {" >> $NFT_METADATA_PATH
echo "     \"$ASSET_NAME\": {" >> $NFT_METADATA_PATH
echo "     \"descripption\": \"$DESCRIPTION\"," >> $NFT_METADATA_PATH
echo "     \"image\": \"$IMAGE\"" >> $NFT_METADATA_PATH
echo "   }" >> $NFT_METADATA_PATH
echo "  }" >> $NFT_METADATA_PATH
echo " }" >> $NFT_METADATA_PATH
echo "}" >> $NFT_METADATA_PATH

# Build the transaction
TOKEN_SUPPLY=1
ASSET_NAME_HEX=$(echo -n $ASSET_NAME | xxd -ps | tr -d '\n')
MINT_ADDR=$(cat /users/$(whoami)/testnet/priv/wallet/Lola/lola.addr)
UTXO_IN=98b172afe15ce5b0efb28bd604431cdd23f8351f90271415f86e782087337872#1
UTXO_IN_AMOUNT=9500000000
MIN_ADA_VALUE=2000000
 
cardano-cli transaction build \
--tx-in $UTXO_IN \
--tx-out $MINT_ADDR+$MIN_ADA_VALUE+"$TOKEN_SUPPLY $POLICY_ID.$ASSET_NAME_HEX" \
--change-address $MINT_ADDR \
--mint "$TOKEN_SUPPLY $POLICY_ID.$ASSET_NAME_HEX" \
--minting-script-file policy/nft.script \
--metadata-json-file metadata/nft.metadata \
--invalid-hereafter $TIME_CONSTRAINT \
--witness-override 2 \
--testnet-magic 1 \
--out-file tx-files/nftMintTx.raw

# Sign the transaction
cardano-cli transaction sign \
--tx-body-file tx-files/nftMintTx.raw \
--signing-key-file /users/$(whoami)/testnet/priv/wallet/Lola/lola.addr.skey \
--signing-key-file priv/NFT/nft.skey \
--out-file tx-files/nftMintTx.signed

# Submi the transaction
cardano-cli transaction submit \
--tx-file tx-files/nftMintTx.signed \
--testnet-magic 1

Estimated transaction fee: Lovelace 187061
Transaction successfully submitted.


### Get the on-chain data of the asset

In [116]:
%%bash 
cardano-cli query utxo \
--address $(cat /users/$(whoami)/testnet/priv/wallet/Lola/lola.addr) \
--testnet-magic 1

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8b1cf6cdc93af09552614ee717259d07e38624a7d1afad2c39552510e0a70f44     0        9497812939 lovelace + TxOutDatumNone
8b1cf6cdc93af09552614ee717259d07e38624a7d1afad2c39552510e0a70f44     1        2000000 lovelace + 1 ffc46973f409fe915cafed2c5cd6a0d93d4cc4ee130d86c5eabef4f7.3344434e33393836 + TxOutDatumNone


In [108]:
%%bash
echo $(cat policy/nft.policyid)$(echo -n "3DCN3986" | xxd -ps | tr -d '\n')

ffc46973f409fe915cafed2c5cd6a0d93d4cc4ee130d86c5eabef4f73344434e33393836


In [118]:
from cardano_explorer import blockfrost_api as bf

preprod= bf.Auth("preprod")
preprod.specific_asset("ffc46973f409fe915cafed2c5cd6a0d93d4cc4ee130d86c5eabef4f73344434e33393836")

Exception: [ERROR 404] Not Found (https://cardano-preprod.blockfrost.io/api/v0/assets/ffc46973f409fe915cafed2c5cd6a0d93d4cc4ee130d86c5eabef4f73344434e33393836). The requested component has not been found.

### 2) Update the NFT metadata

To update the metadata of the asset, you have to mint a second NFT with updated metadata into your own wallet 
and then burn it immediately. 

In [ ]:
# Update the metadata
NFT_ID=721
POLICY_ID=$(cat policy/nft.policyid)
ASSET_NAME="3DClayNation3986"
IMAGE="ipfs://QmVYfnHBhmpVKjAHRpxtwt6XThnafKW37tFtxvCbmHGjzN"
DESCRIPTION="3D render of the clay nation NFT 3986 generate with DALL-E-2 AI"
NFT_METADATA_PATH=metadata/nft.metadata

echo "{" >> $NFT_METADATA_PATH
echo "  \"$NFT_ID\": {" >> $NFT_METADATA_PATH
echo "    \"$POLICY_ID\": {" >> $NFT_METADATA_PATH
echo "     \"$ASSET_NAME\": {" >> $NFT_METADATA_PATH
echo "     \"descripption\": \"$DESCRIPTION\"," >> $NFT_METADATA_PATH
echo "     \"image\": \"$IMAGE\"," >> $NFT_METADATA_PATH
echo "     \"mediaType\": \"image/png\"" >> $NFT_METADATA_PATH
echo "   }" >> $NFT_METADATA_PATH
echo "  }" >> $NFT_METADATA_PATH
echo " }" >> $NFT_METADATA_PATH
echo "}" >> $NFT_METADATA_PATH

# Build the transaction
TOKEN_SUPPLY=1
ASSET_NAME_HEX=$(echo -n $ASSET_NAME | xxd -ps | tr -d '\n')
MINT_ADDR=$(cat /users/$(whoami)/testnet/priv/wallet/Drake/drake.addr)
UTXO_IN=642291a28291970444019cc8186536582871bd602609c2048c0f58274e255b89#0
UTXO_IN_AMOUNT=95631554
MIN_ADA_VALUE=2000000
 
cardano-cli transaction build \
--tx-in $UTXO_IN \
--tx-out $MINT_ADDR+$MIN_ADA_VALUE+"$TOKEN_SUPPLY $POLICY_ID.$ASSET_NAME_HEX" \
--change-address $MINT_ADDR \
--mint "$TOKEN_SUPPLY $POLICY_ID.$ASSET_NAME_HEX" \
--minting-script-file policy/nft.script \
--metadata-json-file metadata/nft.metadata \
--invalid-hereafter $TIME_CONSTRAINT \
--witness-override 2 \
--testnet-magic 1 \
--out-file tx-files/nftMintTx.raw

# Sign the transaction
cardano-cli transaction sign \
--tx-body-file tx-files/nftMintTx.raw \
--signing-key-file /users/$(whoami)/testnet/priv/wallet/Drake/drake.addr.skey \
--signing-key-file priv/NFT/nft.skey \
--out-file tx-files/nftMintTx.signed

# Submi the transaction
cardano-cli transaction submit \
--tx-file tx-files/nftMintTx.signed \
--testnet-magic 1

### 3) Burn the NFT

In [ ]:
# Update the metadata
NFT_ID=721
POLICY_ID=$(cat policy/nft.policyid)
ASSET_NAME="3DClayNation3986"

# Build the transaction
TOKEN_SUPPLY=1
ASSET_NAME_HEX=$(echo -n $ASSET_NAME | xxd -ps | tr -d '\n')
MINT_ADDR=$(cat /users/$(whoami)/testnet/priv/wallet/Drake/drake.addr)
UTXO_IN=642291a28291970444019cc8186536582871bd602609c2048c0f58274e255b89#0
UTXO_IN_AMOUNT=95631554
MIN_ADA_VALUE=2000000
 
cardano-cli transaction build \
--tx-in $UTXO_IN \
--tx-out $MINT_ADDR+$MIN_ADA_VALUE+"$TOKEN_SUPPLY $POLICY_ID.$ASSET_NAME_HEX" \
--change-address $MINT_ADDR \
--mint "-$TOKEN_SUPPLY $POLICY_ID.$ASSET_NAME_HEX" \
--minting-script-file policy/nft.script \
--metadata-json-file metadata/nft.metadata \
--invalid-hereafter $TIME_CONSTRAINT \
--witness-override 2 \
--testnet-magic 1 \
--out-file tx-files/nftMintTx.raw

# Sign the transaction
cardano-cli transaction sign \
--tx-body-file tx-files/nftMintTx.raw \
--signing-key-file /users/$(whoami)/testnet/priv/wallet/Drake/drake.addr.skey \
--signing-key-file priv/NFT/nft.skey \
--out-file tx-files/nftMintTx.signed

# Submi the transaction
cardano-cli transaction submit \
--tx-file tx-files/nftMintTx.signed \
--testnet-magic 1